In [1]:
import pandas as pd
from sacremoses import MosesTokenizer
import string
import emoji

In [2]:
mtk = MosesTokenizer()
emoji_regex = emoji.get_emoji_regexp()

In [3]:
train = pd.read_csv('train_set.csv')
val = pd.read_csv('val_set.csv')
train

,review,rating
0,Package price already received fast delivery m...,4
1,Goods have to be in good condition. Sellsr fas...,4
2,kapaditasnya passable bgt .. thank God,4
3,Awesome awesome quality of goods shipped spee...,5
4,Good bye but have not I go to hopefully cut no...,4
...,...,...
117443,Motifny bgs2 who ordern pertm,3
117444,Good value for money,4
117445,His legs to look very neat that goes on foot.,4
117446,BRG ok..cantik lipez.,5


In [4]:
train['review_clean'] = train['review'].str.lower()
train.sample(5)

,review,rating,review_clean
89368,"Its products are passable, but slow response. ...",1,"its products are passable, but slow response. ..."
98885,"Packets have been received, thank you .. but I...",3,"packets have been received, thank you .. but i..."
80026,Awesome awesome quality of goods and services,5,awesome awesome quality of goods and services
36839,Get rich but pretty satisfied ủa fom too tight...,3,get rich but pretty satisfied ủa fom too tight...
64933,Delivery speed is good.,3,delivery speed is good.


In [5]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

- Punctuations that likely won't impact the sentiment `+*`
- Remove hashtag `#[a-z]+`
- Remove mention `@[a-z]+`
- Remove hyperlinks `http.+`


In [6]:
train[train['review_clean'].str.contains(r"\%")]

,review,rating,review_clean
299,Color e nice. Real pict 99%. If you want to bu...,5,color e nice. real pict 99%. if you want to bu...
375,Will order again 100% Original,5,will order again 100% original
403,Yes. He also came :). I opened the battery to ...,3,yes. he also came :). i opened the battery to ...
1127,Small plates to Forum But does not indicate ho...,3,small plates to forum but does not indicate ho...
1844,Prudential Indonesia together with Junior Achi...,5,prudential indonesia together with junior achi...
...,...,...,...
113561,Excellent quality Very accommodating seller W...,5,excellent quality very accommodating seller w...
115405,extremely fast delivery. i read some of the re...,5,extremely fast delivery. i read some of the re...
115546,Item 100% originu. Packing safe. Sellers could...,5,item 100% originu. packing safe. sellers could...
115608,"Unfortunately, probably like everyone else ......",2,"unfortunately, probably like everyone else ......"


In [7]:
train['review_clean'] = train['review_clean'].str.replace(r'’', '\'')
train['review_clean'] = train['review_clean'].str.replace(r'#[a-z]+', ' ')
train['review_clean'] = train['review_clean'].str.replace(r'@[a-z]+', ' ')
train['review_clean'] = train['review_clean'].str.replace(r'[\+\*=@#<>\(\)\[\]\^_]+', ' ')
train['review_clean'] = train['review_clean'].str.replace(r'[,!?;.:]+', '. ')
train

,review,rating,review_clean
0,Package price already received fast delivery m...,4,package price already received fast delivery m...
1,Goods have to be in good condition. Sellsr fas...,4,goods have to be in good condition. sellsr fa...
2,kapaditasnya passable bgt .. thank God,4,kapaditasnya passable bgt . thank god
3,Awesome awesome quality of goods shipped spee...,5,awesome awesome quality of goods shipped spee...
4,Good bye but have not I go to hopefully cut no...,4,good bye but have not i go to hopefully cut no...
...,...,...,...
117443,Motifny bgs2 who ordern pertm,3,motifny bgs2 who ordern pertm
117444,Good value for money,4,good value for money
117445,His legs to look very neat that goes on foot.,4,his legs to look very neat that goes on foot.
117446,BRG ok..cantik lipez.,5,brg ok. cantik lipez.


In [8]:
train['review_clean'] = train['review_clean'].apply(mtk.tokenize)
train

,review,rating,review_clean
0,Package price already received fast delivery m...,4,"[package, price, already, received, fast, deli..."
1,Goods have to be in good condition. Sellsr fas...,4,"[goods, have, to, be, in, good, condition., se..."
2,kapaditasnya passable bgt .. thank God,4,"[kapaditasnya, passable, bgt, ., thank, god]"
3,Awesome awesome quality of goods shipped spee...,5,"[awesome, awesome, quality, of, goods, shipped..."
4,Good bye but have not I go to hopefully cut no...,4,"[good, bye, but, have, not, i, go, to, hopeful..."
...,...,...,...
117443,Motifny bgs2 who ordern pertm,3,"[motifny, bgs2, who, ordern, pertm]"
117444,Good value for money,4,"[good, value, for, money]"
117445,His legs to look very neat that goes on foot.,4,"[his, legs, to, look, very, neat, that, goes, ..."
117446,BRG ok..cantik lipez.,5,"[brg, ok., cantik, lipez, .]"


In [9]:
train.sample(10)

,review,rating,review_clean
5929,Shop packaged boring. Dented distortion of all...,1,"[shop, packaged, boring., dented, distortion, ..."
74590,Response From good seller SIPP,4,"[response, from, good, seller, sipp]"
36127,order items until 03/18/19 11/03/19 Although t...,2,"[order, items, until, 03, /, 18, /, 19, 11, /,..."
46375,"Fast delivery Packing sure best performance, b...",4,"[fast, delivery, packing, sure, best, performa..."
29802,Good value for money Good product quality,5,"[good, value, for, money, good, product, quality]"
109250,Very memuaskn mbak Mksih,4,"[very, memuaskn, mbak, mksih]"
67433,Good quality good price everything is complete...,2,"[good, quality, good, price, everything, is, c..."
43527,don't like order from this shop,1,"[don, &apos;t, like, order, from, this, shop]"
46867,I'm not certain ... its nice fabric. This arm ...,3,"[i, &apos;m, not, certain, ., its, nice, fabri..."
92766,cpt passable delivery and good packing. good r...,3,"[cpt, passable, delivery, and, good, packing.,..."


In [10]:
def clean(series):
    series = series.str.lower()

    series = series.str.replace(r'’', '\'')
    series = series.str.replace(r'#[a-z]+', ' ')
    series = series.str.replace(r'@[a-z]+', ' ')
    series = series.str.replace(r'[\+\*=@#<>\(\)\[\]\^_]+', ' ')
    series = series.str.replace(r'[,!?;.:]+', '. ')

    series = series.apply(mtk.tokenize)

    return series

val['review_clean'] = clean(val['review'])
val

,review,rating,review_clean
0,But I have put a call saying I'm not hearing it.,1,"[but, i, have, put, a, call, saying, i, &apos;..."
1,"Excellent product quality, use makeup remover ...",5,"[excellent, product, quality., use, makeup, re..."
2,good stuff as reply expected pengirimanya also...,5,"[good, stuff, as, reply, expected, pengirimany..."
3,"Fast delivery, goods accordingly. Not mengecaw...",3,"[fast, delivery., goods, accordingly., not, me..."
4,"Thank you very much min. Payetnya beautiful, g...",5,"[thank, you, very, much, min., payetnya, beaut..."
...,...,...,...
29358,Speed ​​of delivery is good,1,"[speed, ​, ​, of, delivery, is, good]"
29359,the second time the order has reached prodak l...,3,"[the, second, time, the, order, has, reached, ..."
29360,Item shipped immediately !!! Will order again :),5,"[item, shipped, immediately, ., will, order, a..."
29361,The product quality is excellent. The origina...,4,"[the, product, quality, is, excellent., the, o..."


In [14]:
def export_fasttext(tokens, rating):
    return '__label__' + rating.astype(str) + ' ' + tokens.str.join(' ')

train['fasttext'] = export_fasttext(train['review_clean'], train['rating'])
val['fasttext'] = export_fasttext(val['review_clean'], val['rating'])
train

,review,rating,review_clean,fasttext
0,Package price already received fast delivery m...,4,"[package, price, already, received, fast, deli...",__label__4 package price already received fast...
1,Goods have to be in good condition. Sellsr fas...,4,"[goods, have, to, be, in, good, condition., se...",__label__4 goods have to be in good condition....
2,kapaditasnya passable bgt .. thank God,4,"[kapaditasnya, passable, bgt, ., thank, god]",__label__4 kapaditasnya passable bgt . thank god
3,Awesome awesome quality of goods shipped spee...,5,"[awesome, awesome, quality, of, goods, shipped...",__label__5 awesome awesome quality of goods sh...
4,Good bye but have not I go to hopefully cut no...,4,"[good, bye, but, have, not, i, go, to, hopeful...",__label__4 good bye but have not i go to hopef...
...,...,...,...,...
117443,Motifny bgs2 who ordern pertm,3,"[motifny, bgs2, who, ordern, pertm]",__label__3 motifny bgs2 who ordern pertm
117444,Good value for money,4,"[good, value, for, money]",__label__4 good value for money
117445,His legs to look very neat that goes on foot.,4,"[his, legs, to, look, very, neat, that, goes, ...",__label__4 his legs to look very neat that goe...
117446,BRG ok..cantik lipez.,5,"[brg, ok., cantik, lipez, .]",__label__5 brg ok. cantik lipez .


In [15]:
train['fasttext'].to_csv('train_fasttext.csv', index=False, header=False)
val['fasttext'].to_csv('val_fasttext.csv', index=False, header=False)